In [1]:
%%capture
!pip install pyspark
!pip install findspark
!pip install pyngrok

In [26]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [3]:
spark = SparkSession.builder \
        .appName('testColab') \
        .getOrCreate()

In [4]:
from pyngrok import ngrok, conf
import getpass

print("Enter your authtoken, which can be copied "
"from https://dashboard.ngrok.com/get-started/your-authtoken")
conf.get_default().auth_token = getpass.getpass()

ui_port = 4040
public_url = ngrok.connect(ui_port).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{ui_port}\"")

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken
··········


 * ngrok tunnel "https://d23c-34-73-171-104.ngrok-free.app" -> "http://127.0.0.1:4040"


In [7]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

--2025-03-08 08:18:42--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 65.8.245.178, 65.8.245.171, 65.8.245.50, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|65.8.245.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-10.parquet’

yellow_tripdata_202 100%[===================>]  61.36M  40.0MB/s    in 1.5s    

2025-03-08 08:18:43 (40.0 MB/s) - ‘yellow_tripdata_2024-10.parquet’ saved [64346071/64346071]



In [9]:
df = spark.read.parquet("yellow_tripdata_2024-10.parquet")

In [12]:
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [50]:
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

--2025-03-08 08:57:11--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 65.8.245.178, 65.8.245.51, 65.8.245.50, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|65.8.245.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘taxi_zone_lookup.csv’

taxi_zone_lookup.cs 100%[===================>]  12.04K  --.-KB/s    in 0s      

2025-03-08 08:57:11 (190 MB/s) - ‘taxi_zone_lookup.csv’ saved [12331/12331]



In [ ]:
df.createOrReplaceTempView("taxi_data")
df = spark.read.parquet("repartitioned_data/")

In [64]:
df_2 = spark.read.csv("taxi_zone_lookup.csv", header=True)
df_2.createOrReplaceTempView("taxi_zone_lookup")

In [55]:
df_2.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



#### Question 1

In [5]:
spark.version

'3.5.5'

#### Question 2

In [13]:
df.repartition(4).write.parquet("repartitioned_data/")

In [14]:
!ls -lh repartitioned_data/

total 93M
-rw-r--r-- 1 root root 24M Mar  8 08:22 part-00000-9c531ece-1246-49cd-a5d1-07a73ec4f2cf-c000.snappy.parquet
-rw-r--r-- 1 root root 24M Mar  8 08:22 part-00001-9c531ece-1246-49cd-a5d1-07a73ec4f2cf-c000.snappy.parquet
-rw-r--r-- 1 root root 24M Mar  8 08:22 part-00002-9c531ece-1246-49cd-a5d1-07a73ec4f2cf-c000.snappy.parquet
-rw-r--r-- 1 root root 24M Mar  8 08:22 part-00003-9c531ece-1246-49cd-a5d1-07a73ec4f2cf-c000.snappy.parquet
-rw-r--r-- 1 root root   0 Mar  8 08:22 _SUCCESS


#### Question 3

In [16]:
df.filter((df.tpep_pickup_datetime >= "2024-10-15") & (df.tpep_pickup_datetime < "2024-10-16")).count()

128893

#### Question 4

In [48]:
new_df = df.withColumn('distance_hours', (unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime"))/3600)

In [49]:
new_df.select(max(col("distance_hours"))).show()

+-------------------+
|max(distance_hours)|
+-------------------+
| 162.61777777777777|
+-------------------+



#### Question 5
4040

#### Question 6

In [63]:
spark.sql("""
  select
    t2.Zone,
    count(1) as cnt
  from taxi_data as t1
  left join taxi_zone_lookup as t2
  on t1.PULocationID = t2.LocationID
  group by 1
  order by 2
  limit 1
""").show(truncate=False)

+---------------------------------------------+---+
|Zone                                         |cnt|
+---------------------------------------------+---+
|Governor's Island/Ellis Island/Liberty Island|1  |
+---------------------------------------------+---+

